# Indexación jerárquica

Hasta ahora nos hemos centrado principalmente en datos unidimensionales y bidimensionales, almacenados en objetos Pandas ``Series`` y ``DataFrame``, respectivamente.
A menudo es útil ir más allá y almacenar datos de mayor dimensión, es decir, datos indexados por más de una o dos claves.

Aunque Pandas proporciona objetos ``Panel`` y ``Panel4D`` que manejan de forma nativa datos tridimensionales y cuatridimensionales, un patrón mucho más común en la práctica es hacer uso de la *indización jerárquica* (también conocida como *multiindización*) para incorporar múltiples *niveles de índice* dentro de un único índice.
De este modo, los datos de mayor dimensión pueden representarse de forma compacta dentro de los conocidos objetos unidimensionales ``Series`` y bidimensionales ``DataFrame``.

En esta sección, exploraremos la creación directa de objetos ``MultiIndex``, las consideraciones a tener en cuenta a la hora de indexar, trocear y calcular estadísticas a través de datos indexados de forma múltiple, y rutinas útiles para convertir entre representaciones simples y jerárquicamente indexadas de tus datos.

Comenzaremos con las importaciones estándar:

In [1]:
import pandas as pd
import numpy as np

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_19764\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## A Multiply Indexed Series

Empecemos por considerar cómo podríamos representar datos bidimensionales dentro de una ``Serie`` unidimensional.
Para concretar, consideraremos una serie de datos donde cada punto tiene un carácter y una clave numérica.

### The bad way

Supongamos que quieres rastrear datos sobre estados de dos años diferentes.
Usando las herramientas de Pandas que ya hemos cubierto, podrías estar tentado a usar simplemente tuplas de Python como claves:

In [59]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

Con este esquema de indexación, puede indexar o trocear directamente las series basándose en este índice múltiple:

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

Pero la comodidad termina ahí. Por ejemplo, si necesitas seleccionar todos los valores a partir de 2010, tendrás que hacer algunas operaciones complicadas (y potencialmente lentas) para conseguirlo:

In [ ]:
pop[[i for i in pop.index if i[1] == 2010]]

Esto produce el resultado deseado, pero no es tan limpio (o tan eficiente para grandes conjuntos de datos) como la sintaxis de corte que tanto nos gusta en Pandas.

### The Better Way: Pandas MultiIndex
Afortunadamente, Pandas proporciona una forma mejor.
Nuestra indexación basada en tuplas es esencialmente un multiíndice rudimentario, y el tipo ``MultiIndex`` de Pandas nos proporciona el tipo de operaciones que deseamos tener.
Podemos crear un multiíndice a partir de las tuplas de la siguiente manera:

In [ ]:
index = pd.MultiIndex.from_tuples(index)
index

Observe que ``MultiIndex`` contiene varios *niveles* de indexación, en este caso, los nombres de los estados y los años, así como varias *etiquetas* para cada punto de datos que codifican estos niveles.

Si volvemos a indexar nuestra serie con este ``MultiIndex``, veremos la representación jerárquica de los datos:

In [61]:
pop = pop.reindex(index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

Aquí, las dos primeras columnas de la representación ``Series`` muestran los valores de los índices múltiples, mientras que la tercera columna muestra los datos..

Observe que faltan algunas entradas en la primera columna: en esta representación de índices múltiples, cualquier entrada en blanco indica el mismo valor que la línea situada encima.

Ahora, para acceder a todos los datos para los que el segundo índice es 2010, podemos simplemente utilizar la notación de Pandas slicing:

In [ ]:
pop[:, 2010]

In [43]:
# Ejemplo real: Población de Cuenca
cuenca = pd.read_csv("data/cuenca.csv", dtype={"Total":str})


In [5]:
cuenca.head()

,Unnamed: 0,Municipios,Sexo,Periodo,Total
0,0,16,Total,2023,197.139
1,1,16,Total,2022,195.215
2,2,16,Total,2021,195.516
3,3,16,Total,2020,196.139
4,4,16,Total,2019,196.329


In [6]:
cuenca.tail()

,Unnamed: 0,Municipios,Sexo,Periodo,Total
20071,20071,16280,Mujeres,2000,147.0
20072,20072,16280,Mujeres,1999,150.0
20073,20073,16280,Mujeres,1998,148.0
20074,20074,16280,Mujeres,1997,NaN
20075,20075,16280,Mujeres,1996,153.0


In [41]:
cuenca_original = pd.read_csv("data/cuenca.csv", dtype={"Total":str})

In [44]:
cuenca.drop(columns=["Unnamed: 0"], inplace = True,errors="ignore")
cuenca.head(100)

,Municipios,Sexo,Periodo,Total
0,16,Total,2023,197.139
1,16,Total,2022,195.215
2,16,Total,2021,195.516
3,16,Total,2020,196.139
4,16,Total,2019,196.329
...,...,...,...,...
95,16001,Total,2012,81.0
96,16001,Total,2011,76.0
97,16001,Total,2010,73.0
98,16001,Total,2009,80.0


In [45]:
cuenca.columns = ["municipios", "sexo", "anno", "total"]

In [46]:
cuenca.head()

,municipios,sexo,anno,total
0,16,Total,2023,197.139
1,16,Total,2022,195.215
2,16,Total,2021,195.516
3,16,Total,2020,196.139
4,16,Total,2019,196.329


In [47]:
cuenca['total'] = cuenca['total'].str.replace(".", "").astype(float)

In [48]:
cuenca.head()

,municipios,sexo,anno,total
0,16,Total,2023,197139.0
1,16,Total,2022,195215.0
2,16,Total,2021,195516.0
3,16,Total,2020,196139.0
4,16,Total,2019,196329.0


In [51]:
cuenca = cuenca.set_index(["municipios", "sexo", "anno"])

KeyError: "None of ['municipios', 'sexo', 'anno'] are in the columns"

In [52]:
cuenca.head(10)

total
municipios sexo  anno          
16         Total 2023  197139.0
                 2022  195215.0
                 2021  195516.0
                 2020  196139.0
                 2019  196329.0
                 2018  197222.0
                 2017  198718.0
                 2016  201071.0
                 2015  203841.0
                 2014  207449.0

In [53]:
cuenca.tail(10)

total
municipios sexo    anno        
16280      Mujeres 2005  1280.0
                   2004  1270.0
                   2003  1290.0
                   2002  1320.0
                   2001  1370.0
                   2000  1470.0
                   1999  1500.0
                   1998  1480.0
                   1997     NaN
                   1996  1530.0

In [54]:
cuenca_series = pd.Series(cuenca['total'])

In [55]:
cuenca_series

municipios  sexo     anno
16          Total    2023    197139.0
                     2022    195215.0
                     2021    195516.0
                     2020    196139.0
                     2019    196329.0
                               ...   
16280       Mujeres  2000      1470.0
                     1999      1500.0
                     1998      1480.0
                     1997         NaN
                     1996      1530.0
Name: total, Length: 20076, dtype: float64

El resultado es un array indexado individualmente sólo con las claves que nos interesan.
Esta sintaxis es mucho más cómoda (¡y la operación es mucho más eficiente!) que la solución casera de indexación múltiple basada en tuplas con la que empezamos.
A continuación analizaremos este tipo de operaciones de indexación en datos indexados jerárquicamente.

### MultiIndex como dimensión extra

Puedes notar algo más aquí: fácilmente podríamos haber almacenado los mismos datos usando un simple ``DataFrame`` con etiquetas de índice y columna.
De hecho, Pandas está construido con esta equivalencia en mente. El método ``unstack()`` convertirá rápidamente una ``Serie`` con índices múltiples en un ``DataFrame`` con índices convencionales:

In [62]:
pop_df = pop.unstack()
pop_df

ValueError: index must be a MultiIndex to unstack, <class 'pandas.core.indexes.base.Index'> was passed

In [56]:
# Poco conveniente para manejar
cs_unstack = cuenca_series.unstack()

In [57]:
cs_unstack

anno                    1996  1997      1998      1999      2000      2001  \
municipios sexo                                                              
16         Hombres  100387.0   NaN   99031.0  100088.0  100126.0   10063.0   
           Mujeres  101325.0   NaN  100055.0  100875.0  100927.0  100896.0   
           Total    201712.0   NaN  199086.0  200963.0  201053.0  201526.0   
16001      Hombres     500.0   NaN     480.0     460.0     460.0     450.0   
           Mujeres     430.0   NaN     400.0     390.0     390.0     380.0   
...                      ...   ...       ...       ...       ...       ...   
16909      Mujeres    5000.0   NaN    4870.0    5640.0    5150.0    5070.0   
           Total      9840.0   NaN    9580.0    1119.0    1021.0    1012.0   
16910      Hombres     750.0   NaN     720.0     690.0     680.0     650.0   
           Mujeres     660.0   NaN     590.0     560.0     550.0     590.0   
           Total      1410.0   NaN    1310.0    1250.0    1230.0    1240.0   

anno                    2002      2003      2004      2005  ...      2014  \
municipios sexo                                             ...             
16         Hombres  101048.0   10204.0  102947.0  104912.0  ...  104586.0   
           Mujeres  100566.0  100942.0  101599.0  103062.0  ...  102863.0   
           Total    201614.0  202982.0  204546.0  207974.0  ...  207449.0   
16001      Hombres     490.0     490.0     480.0     480.0  ...     500.0   
           Mujeres     360.0     400.0     400.0     360.0  ...     300.0   
...                      ...       ...       ...       ...  ...       ...   
16909      Mujeres    4980.0    4900.0    4680.0    4460.0  ...    4060.0   
           Total      9910.0    9680.0    9360.0    9010.0  ...    8290.0   
16910      Hombres     650.0     620.0     610.0     610.0  ...     550.0   
           Mujeres     570.0     540.0     530.0     500.0  ...     460.0   
           Total      1220.0    1160.0    1140.0    1110.0  ...    1010.0   

anno                    2015      2016      2017      2018      2019  \
municipios sexo                                                        
16         Hombres  102583.0  101097.0   99821.0   98999.0   98542.0   
           Mujeres  101258.0   99974.0   98897.0   98223.0   97787.0   
           Total    203841.0  201071.0  198718.0  197222.0  196329.0   
16001      Hombres     490.0     480.0     400.0     390.0     400.0   
           Mujeres     280.0     290.0     260.0     250.0     250.0   
...                      ...       ...       ...       ...       ...   
16909      Mujeres    3930.0    3910.0    3760.0    3670.0    3540.0   
           Total      8020.0    7910.0    7760.0    7560.0    7320.0   
16910      Hombres     570.0     510.0     480.0     470.0     440.0   
           Mujeres     470.0     410.0     400.0     350.0     330.0   
           Total      1040.0     920.0     880.0     820.0     770.0   

anno                    2020      2021      2022      2023  
municipios sexo                                             
16         Hombres   98430.0   98118.0   98029.0   99137.0  
           Mujeres   97709.0   97398.0   97186.0   98002.0  
           Total    196139.0  195516.0  195215.0  197139.0  
16001      Hombres     370.0     350.0     370.0     380.0  
           Mujeres     240.0     220.0     230.0     230.0  
...                      ...       ...       ...       ...  
16909      Mujeres    3350.0    3210.0    3340.0    3320.0  
           Total      7000.0    6810.0    6960.0    6980.0  
16910      Hombres     430.0     420.0     440.0     450.0  
           Mujeres     320.0     310.0     350.0     330.0  
           Total       750.0     730.0     790.0     780.0  

[717 rows x 28 columns]

Naturalmente, el método ``stack()`` proporciona la operación contraria:

In [ ]:
pop_df.stack()

In [ ]:
cs_unstack.stack()

Además, todas las ufuncs y otras funcionalidades discutidas en [Operating on Data in Pandas](03_Operations-in-Pandas.ipynb) funcionan también con índices jerárquicos.

Aquí calculamos la fracción de personas menores de 18 años por año, dados los datos anteriores:

In [ ]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

In [ ]:
cuenca['ciudad'] = cuenca['total'] > 10000

In [ ]:
cuenca.head()

Esto nos permite manipular y explorar fácil y rápidamente incluso datos de alta dimensión.

## Métodos de creación de MultiIndex

La forma más directa de construir una ``Serie`` o un ``DataFrame`` multiíndice es simplemente pasar una lista de dos o más matrices de índices al constructor. Por ejemplo

In [ ]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

El trabajo de creación del ``MultiIndex`` se realiza en segundo plano.

Del mismo modo, si pasas un diccionario con tuplas apropiadas como claves, Pandas lo reconocerá automáticamente y usará un ``MultiIndex`` por defecto:

In [ ]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

Sin embargo, a veces es útil crear explícitamente un ``MultiIndex``; veremos un par de estos métodos aquí.

### Constructores explícitos MultiIndex

Para una mayor flexibilidad en la construcción del índice, puedes utilizar los métodos constructores de la clase ``pd.MultiIndex``.
Por ejemplo, como hicimos antes, puedes construir el ``MultiIndex`` a partir de una simple lista de arrays con los valores del índice dentro de cada nivel:

In [ ]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

Se puede construir a partir de una lista de tuplas que den los valores de índice múltiple de cada punto:

In [ ]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

Incluso se puede construir a partir de un producto cartesiano de índices simples:

In [ ]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

Del mismo modo, puedes construir el ``MultiIndex`` directamente utilizando su codificación interna pasando ``levels`` (una lista de listas que contienen los valores de índice disponibles para cada nivel) y ``codes`` (una lista de listas que hacen referencia a estas etiquetas):

In [ ]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

Cualquiera de estos objetos puede pasarse como argumento ``index`` al crear una ``Series`` o un ``Dataframe``, o pasarse al método ``reindex`` de una ``Series`` o un ``DataFrame`` ya existentes.

Como lo más habitual es mediante la lectura de fuentes de datos externas, tenemos el método `set_index()`

In [ ]:
cuenca_original = pd.read_csv("data/cuenca.csv", dtype={"Total":str})

In [ ]:
cuenca_original

In [ ]:
cuenca_original.set_index(["Municipios", "Sexo", "Periodo"])

In [ ]:
cuenca_original = pd.read_csv("data/cuenca.csv", dtype={"Total":str}, index_col =["Municipios", "Sexo", "Periodo"])

In [ ]:
cuenca_original.head()

### Nombres de niveles MultiIndex

A veces es conveniente nombrar los niveles del ``MultiIndex``.
Esto se puede conseguir pasando el argumento ``names`` a cualquiera de los constructores de ``MultiIndex``, o estableciendo el atributo ``names`` del índice a posteriori:

In [ ]:
pop.index.names = ['state', 'year']
pop

Con conjuntos de datos más complejos, puede ser una forma útil de seguir el significado de varios valores de índice.

### MultiIndex para columnas

En un ``DataFrame``, las filas y columnas son completamente simétricas, y al igual que las filas pueden tener múltiples niveles de índices, las columnas también pueden tener múltiples niveles.

Considere lo siguiente, que es una maqueta de algunos datos médicos:

In [ ]:
# índices jerárquicos y columnas
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# simular algunos datos
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# crear el DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

En este caso, la indexación múltiple de filas y columnas puede resultar útil en algunas ocasiones. No obstante, suele ser más cómodo utilizar modelos de DS usando `DataFrame` sin índices


Se trata fundamentalmente de datos cuatridimensionales, en los que las dimensiones son el sujeto, el tipo de medición, el año y el número de visita.
De este modo podemos, por ejemplo, indexar la columna superior por el nombre de la persona y obtener un ``DataFrame`` completo que contenga sólo la información de esa persona:

In [ ]:
health_data['Guido']

In [ ]:
cuenca_original = pd.read_csv("data/cuenca.csv", dtype={"Total":str})

In [ ]:
cuenca_original.set_index(["Municipios", "Sexo", "Periodo"]).unstack()

En el caso de registros complicados que contengan múltiples mediciones etiquetadas a lo largo de múltiples tiempos para muchos sujetos (personas, países, ciudades, etc.), el uso de filas y columnas jerárquicas puede resultar extremadamente cómodo para visualización/manipulación, pero no tanto para tratamiendo en modelos

## Indexación y segmentación de un MultiIndex

Indexar y rebanar en un ``MultiIndex`` está diseñado para ser intuitivo, y ayuda si piensas en los índices como dimensiones añadidas.
Primero veremos la indexación de ``Series`` multiíndices, y después la de ``DataFrame`` multiíndices.

### Multiply indexed Series

Consideremos la ``Serie`` de poblaciones estatales de índice múltiple que vimos anteriormente:

In [ ]:
pop

In [ ]:
cuenca_series.head()

Podemos acceder a elementos individuales indexando con varios términos:

In [ ]:
pop['California', 2000]

In [ ]:
cuenca_series[16, 'Mujeres', 2000]

El ``MultiIndex`` también admite la *indización parcial*, es decir, la indización de sólo uno de los niveles del índice.
El resultado es otra ``Serie``, que mantiene los índices de los niveles inferiores:

In [ ]:
pop['California']

In [ ]:
# En el mismo orden!
cuenca_series[16]

In [ ]:
# En el mismo orden!
cuenca_series['Mujeres']

In [ ]:
# En el mismo orden!
cuenca_series[16, 'Mujeres']

El corte parcial también está disponible, siempre que el ``MultiIndex``:

In [ ]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
index = pd.MultiIndex.from_tuples(index)
pop = pop.reindex(index)


In [ ]:
pop['California':'New York']

In [ ]:
cuenca_series[16200:16280]

Con índices ordenados, se puede realizar una indexación parcial en niveles inferiores pasando una rebanada vacía en el primer índice:

In [ ]:
pop[:, 2000]

In [ ]:
cuenca_series

In [ ]:
cuenca_series[:, 'Total', 2000]

También funcionan otros tipos de indexación y selección (tratados en [Indexación y selección de datos](02_Data-Indexing-and-Selection.ipynb)); por ejemplo, la selección basada en máscaras booleanas:

In [ ]:
pop[pop > 22000000]

In [ ]:
cuenca_series[cuenca_series > 50000]

La selección basada en una indexación elegante también funciona:

In [ ]:
pop[['California', 'Texas']]

### Multiply indexed DataFrames

Un ``DataFrame`` con índices múltiples se comporta de forma similar.
Consideremos nuestro ``DataFrame`` médico de juguete de antes:

In [ ]:
health_data

Recuerde que las columnas son primarias en un ``DataFrame``, y que la sintaxis utilizada para las ``Series`` con índices múltiples se aplica a las columnas.
Por ejemplo, podemos recuperar los datos de la frecuencia cardíaca de Guido con una simple operación:

In [ ]:
health_data['Guido', 'HR']

In [ ]:
cuenca['total']

Además, como en el caso del índice único, podemos utilizar los indexadores ``loc``, ``iloc`` y ``ix`` introducidos en [Indexación y selección de datos](03.02-Indexación y selección de datos.ipynb). Por ejemplo:

In [ ]:
health_data.iloc[:2, :2]

In [ ]:
cuenca.iloc[:2, :2]

In [ ]:
cuenca_unstack = cuenca.unstack()

In [ ]:
cuenca_unstack.head()

Estos indexadores proporcionan una vista similar a un array de los datos bidimensionales subyacentes, pero a cada índice individual de ``loc`` o ``iloc`` se le puede pasar una tupla de múltiples índices. Por ejemplo:

In [ ]:
health_data.loc[:, ('Bob', 'HR')]

In [ ]:
cuenca_unstack.loc[:, ('total', 1996)]

Trabajar con trozos dentro de estas tuplas de índice no es especialmente cómodo; intentar crear un trozo dentro de una tupla provocará un error de sintaxis:

In [ ]:
health_data.loc[(:, 1), (:, 'HR')]

Se podría evitar esto construyendo la porción deseada explícitamente usando la función ``slice()`` de Python, pero una mejor manera en este contexto es usar un objeto ``IndexSlice``, que Pandas proporciona precisamente para esta situación.
Por ejemplo:

In [ ]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

Hay muchas formas de interactuar con los datos de las ``Series`` y los ``DataFrame`` multiíndices, y como ocurre con muchas de las herramientas de este libro, la mejor forma de familiarizarse con ellas es probarlas.

## Rearranging Multi-Indices

Una de las claves para trabajar con datos de índices múltiples es saber cómo transformar los datos de forma eficaz.
Hay una serie de operaciones que conservarán toda la información en el conjunto de datos, pero la reorganizarán a los efectos de diversos cálculos.
Vimos un breve ejemplo de esto en los métodos ``stack()`` y ``unstack()``, pero hay muchas más formas de controlar finamente el reordenamiento de datos entre índices jerárquicos y columnas, y las exploraremos aquí.

### Índices ordenados y no ordenados

Antes mencionamos brevemente una advertencia, pero deberíamos enfatizarla más aquí.
*Muchas de las operaciones de corte ``MultiIndex`` fallarán si el índice no está ordenado.*
Veámoslo aquí.

Empezaremos creando unos simples datos multiíndice en los que los índices *no están lexográficamente ordenados*:

In [ ]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

Si intentamos tomar una porción parcial de este índice, se producirá un error:

In [ ]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

In [ ]:
cuenca_series[16]

In [ ]:
cuenca_series.index

Aunque no queda del todo claro en el mensaje de error, esto se debe a que el MultiIndex no está ordenado.
Por varias razones, los cortes parciales y otras operaciones similares requieren que los niveles del ``MultiIndex`` estén ordenados (es decir, lexográficamente).
Pandas proporciona una serie de rutinas para realizar este tipo de ordenación; algunos ejemplos son los métodos ``sort_index()`` y ``sortlevel()`` del ``DataFrame``.
Aquí usaremos el más simple, ``sort_index()``:

In [ ]:
data = data.sort_index()
data

In [ ]:
cuenca_series = cuenca_series.sort_index()
cuenca_series

In [ ]:
cuenca_series[16]['Hombres':'Mujeres']

Con el índice ordenado de esta forma, el corte parcial funcionará como se espera:

In [ ]:
data['a':'b']

### Índices de apilamiento y desapilamiento

Como vimos brevemente antes, es posible convertir un conjunto de datos de un multiíndice apilado a una representación bidimensional simple, especificando opcionalmente el nivel a utilizar:

In [ ]:
pop.unstack(level=0)

In [ ]:
cuenca_series.unstack(level=0)

In [ ]:
pop.unstack(level=1)

In [ ]:
cuenca_series.unstack(level=1)

In [ ]:
# Muy útil para cambiar filas por columnas
cuenca_series.unstack(level=1).reset_index()


Lo contrario de ``unstack()`` es ``stack()``, que aquí se puede utilizar para recuperar la serie original:

In [ ]:
pop.unstack().stack()

In [ ]:
cuenca_series.unstack().stack()

### Index setting and resetting

Otra forma de reorganizar los datos jerárquicos es convertir las etiquetas de índice en columnas; esto se puede conseguir con el método ``reset_index``.
Al llamar a este método en el diccionario de población se obtendrá un ``DataFrame`` con una columna *state* y *year* que contendrá la información que antes estaba en el índice.
Para mayor claridad, podemos especificar opcionalmente el nombre de los datos para la representación de la columna:

In [ ]:
pop_flat = pop.reset_index(name='population')
pop_flat

In [ ]:
cuenca_series.reset_index("sexo")

A menudo, cuando se trabaja con datos en el mundo real, los datos de entrada en bruto tienen este aspecto y es útil construir un ``MultiIndex`` a partir de los valores de las columnas.
Esto se puede hacer con el método ``set_index`` del ``DataFrame``, que devuelve un ``DataFrame`` con índice múltiple:

In [ ]:
pop_flat.set_index(['state', 'year'])

En la práctica, encuentro que este tipo de reindexación es uno de los patrones más útiles cuando me encuentro con conjuntos de datos del mundo real.

## Agregaciones de datos en multiíndices

Hemos visto anteriormente que Pandas tiene métodos de agregación de datos incorporados, como ``mean()``, ``sum()``, y ``max()``.
Para datos indexados jerárquicamente, se les puede pasar un parámetro ``level`` que controla sobre qué subconjunto de datos se calcula el agregado.

Por ejemplo, volvamos a nuestros datos de salud:

In [ ]:
health_data

Tal vez nos gustaría calcular la media de las mediciones en las dos visitas de cada año. Podemos hacerlo nombrando el nivel del índice que nos gustaría explorar, en este caso el año:

In [ ]:
data_mean = health_data.groupby('year').mean()
data_mean

In [ ]:
cuenca_media = cuenca_series.groupby('anno').mean()
cuenca_media

In [ ]:
cuenca_media = cuenca_series.groupby('sexo').mean()
cuenca_media

Así, en dos líneas, hemos podido encontrar la media de la frecuencia cardiaca y la temperatura medidas entre todos los sujetos en todas las visitas de cada año.
Esta sintaxis es en realidad un atajo a la funcionalidad ``GroupBy``, de la que hablaremos en [Aggregation and Grouping](8_Aggregation-and-Grouping.ipynb).
Aunque éste es un ejemplo de juguete, muchos conjuntos de datos del mundo real tienen una estructura jerárquica similar.

## Aside: Datos de panel

Pandas tiene algunas otras estructuras de datos fundamentales que aún no hemos discutido, a saber, los objetos ``pd.Panel`` y ``pd.Panel4D``.
Se pueden considerar, respectivamente, como generalizaciones tridimensionales y cuatridimensionales de las estructuras (unidimensionales) ``Series`` y (bidimensionales) ``DataFrame``.
Una vez familiarizado con la indexación y manipulación de datos en ``Series`` y ``DataFrame``, ``Panel`` y ``Panel4D`` son relativamente sencillos de utilizar.
En particular, los indexadores  ``loc`` y ``iloc`` discutidos en [Data Indexing and Selection](02_Data-Indexing-and-Selection.ipynb) se extienden fácilmente a estas estructuras de mayor dimensión.

En este texto no trataremos más estas estructuras de panel, ya que en la mayoría de los casos he comprobado que la multiindización es una representación más útil y conceptualmente más sencilla para los datos de mayor dimensión.
Además, los datos de panel son fundamentalmente una representación de datos densos, mientras que la multiindización es fundamentalmente una representación de datos dispersos.
A medida que aumenta el número de dimensiones, la representación densa puede resultar muy ineficiente para la mayoría de los conjuntos de datos del mundo real.
Sin embargo, para aplicaciones especializadas ocasionales, estas estructuras pueden resultar útiles.